In [2]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import dbManagement as dbm

import redCellGUI as rgui

In [6]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [2]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [28]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL020/2023-05-04/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
frame2time had 2 more than suite2p output. This happens sometimes. I don't like it. I think it's because scanimage sends a TTL before starting the frame
ATL020/2023-05-04/702:spks shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:F shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:Fneu shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:redcell shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:iscell shapes: [1233, 421, 1105, 1915, 1341]


In [5]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [12]:
iscell = vrexp.loadS2P('iscell', concatenate=True, checkVariables=False)
redcell = vrexp.loadS2P('redcell', concatenate=True, checkVariables=False)
np.unique(redcell[:,0])

array([0., 1.], dtype=float32)

In [8]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 1.14 s
Wall time: 476 ms


In [9]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [65]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!
# ---------------------
# vrExperimentRegistration
# -- if error, then note there was an error and append an explanation...!!!

In [66]:
vrSession = vre.vrSession('ATL012','2023-03-01','701')

In [68]:
s2pFiles = vrSession.suite2pPath().rglob('*.npy')

for f in s2pFiles: 
    try: 
        np.load(f, allow_pickle=True)
    except:
        print(f)

C:\Users\andrew\Documents\localData\ATL012\2023-03-01\701\suite2p\plane4\F.npy
C:\Users\andrew\Documents\localData\ATL012\2023-03-01\701\suite2p\plane4\Fneu.npy
C:\Users\andrew\Documents\localData\ATL012\2023-03-01\701\suite2p\plane4\F_chan2.npy
C:\Users\andrew\Documents\localData\ATL012\2023-03-01\701\suite2p\plane4\spks.npy


In [102]:
import pandas as pd
mouseName, sessionDate, sessionID = 'ATL022', '2023-03-29', '701'

fieldNames, tableData = vrdb.tableData()
df = pd.DataFrame.from_records(tableData, columns=fieldNames)
record = df[(df['mouseName']==mouseName) 
            & (df['sessionDate'].apply(lambda sd : sd.strftime('%Y-%m-%d'))==sessionDate)
            & (df['sessionID']==int(sessionID))]


In [111]:
record['mouseName'].item()

'ATL022'

In [112]:
record['sessionDate'].dt.strftime('%Y-%m-%d').item()

'2023-03-29'

In [113]:
df = vrdb.getTable()

In [120]:
for i, row in df.iterrows():
    print(type(row))
    print(row['sessionDate'])
    print(row['sessionDate'].strftime('%Y-%m-%d'))
    print(row)
    break

<class 'pandas.core.series.Series'>
2023-01-18 00:00:00
2023-01-18
uSessionID                                                                 1
mouseName                                                             ATL012
sessionDate                                              2023-01-18 00:00:00
sessionID                                                                701
experimentType                                                    Blender VR
experimentID                                                               1
variableGain                                                           False
behavior                                                                True
imaging                                                                 True
faceCamera                                                             False
vrEnvironments                                                             1
headPlateRotation                                                        NaN
numPlanes

In [127]:
record = vrdb.getRecord('ATL012','2023-01-18','701')

In [130]:
record['sessionDate'].strftime('%Y-%m-%d')

'2023-01-18'

In [133]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL022','2023-04-05','702')

\\zaru.cortexlab.net\Subjects\ATL022\2023-04-05\702
C:\Users\andrew\Documents\localData\ATL022\2023-04-05\702


In [69]:
vrdb = dbm.vrDatabase()

In [156]:
vrdb.printRegistrationErrors()
# I fixed ATL012/2023-03-01
#         ATL020/2023-03-24
#         ATL022/2023-03-29


Session ATL020/2023-04-11/701 had error: Cannot fix mismatches when suite2p data is missing!
Session ATL022/2023-04-05/702 had error: cannot reshape array of size 0 into shape (3278,16867)
Session ATL022/2023-04-14/701 had error: In session ATL022/2023-04-14/701, first flips in trial are not all down!!
Session ATL022/2023-04-17/701 had error: In session ATL022/2023-04-17/701, first flips in trial are not all down!!
Session ATL022/2023-05-02/701 had error: Reader needs file name or open file-like object
Session ATL028/2023-08-04/701 had error: Cannot fix mismatches when suite2p data is missing!


In [155]:
# vrdb.registerSessions()
out = vrdb.registerSingleSession('ATL022','2023-04-21','701')

Performing vrExperiment preprocessing for session: ATL022/2023-04-21/701
Cleared oneData from session: ATL022/2023-04-21/701
In session: ATL022/2023-04-21/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
Computing red cell features for ATL022/2023-04-21/701... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
Saving params...
Session ATL022/2023-04-21/701 registered with 1.96 GB oneData.


In [1]:
import fileManagement as fm
fm.s2pTargets('ATL022','2023-04-21','701')

\\zaru.cortexlab.net\Subjects\ATL022\2023-04-21\701
C:\Users\andrew\Documents\localData\ATL022\2023-04-21\701


In [145]:
vrreg = vre.vrExperimentRegistration('ATL020', '2023-03-24', '701')

In [ ]:
# vrdb.updateSuite2pDateTime()
# vrdb.checkS2P(withDatabaseUpdate=True)

In [ ]:
vrdb.printRequiresS2P(needsQC=True)

Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-18/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-23/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-24/703
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-25/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-26/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-27/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-30/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-01/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-02/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-06/701
Database indicates that suit

In [3]:
import PyQt5

In [6]:
dir(PyQt5)

['QtCore',
 'QtGui',
 'QtSvg',
 'QtWidgets',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'sip']